**⚠️ DISCLAIMER** : *Uses open-source price data via yfinance. Not recommended for immediate commercial use. Deploy only after using verified brokerage data and thorough backtesting.*

*INSTALLATIONS*

In [ ]:
!pip install optuna
!pip install ta
!pip install pykalman
!pip install quantstats
!pip install ipywidgets
!pip install IPython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=464d2d3edb8aabad9fde8e5670dd0a19d07088e8cd04c34c46ead7eb04f8ab50
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.5/248.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.5/145.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.1 MB/s eta 0:00:00


IMPORTS

In [ ]:
# CELL 1: Import Libraries and Basic Setup

import yfinance as yf
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import ta
import numpy as np
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from optuna import create_study
from optuna.samplers import TPESampler
import joblib
from xgboost import DMatrix
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from scipy.stats import skew, kurtosis
from tqdm import tqdm
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
import joblib
import os
from datetime import datetime
import logging
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
import requests
from bs4 import BeautifulSoup
import time
from tenacity import retry, stop_after_attempt, wait_exponential
from datetime import date
from pprint import pprint
import json
from numpy import log, polyfit
from sklearn.linear_model import LinearRegression
from sklearn.inspection import permutation_importance
import shap
import warnings
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
import quantstats as qs
import numpy as np
from pykalman import KalmanFilter
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Suppress warnings
warnings.filterwarnings('ignore', category=RuntimeWarning, module='statsmodels\\.regression\\.linear_model')
warnings.filterwarnings('ignore', category=UserWarning)

DATA LOADING , CLEANING

In [ ]:
# CELL 2: Data Loading and Cleaning

def get_nifty50_tickers():
    """
    Modified to use only POWERGRID.NS for testing
    In production, uncomment the full list
    """
    #return ["TATACONSUM.NS"]

      #Full NIFTY 50 list (commented for testing)
    return [
         "ADANIENT.NS", "ADANIPORTS.NS", "ADANIGREEN.NS",
         "ASIANPAINT.NS", "AXISBANK.NS", "BAJFINANCE.NS", "BAJAJFINSV.NS",
         "BPCL.NS", "BRITANNIA.NS", "CIPLA.NS", "COALINDIA.NS", "DIVISLAB.NS",
         "DRREDDY.NS", "EICHERMOT.NS", "GAIL.NS", "GRASIM.NS", "HCLTECH.NS",
         "HDFCBANK.NS", "HEROMOTOCO.NS", "HINDUNILVR.NS",
         "ICICIBANK.NS", "INDUSINDBK.NS", "INFY.NS", "ITC.NS", "JSWSTEEL.NS",
         "KOTAKBANK.NS", "LT.NS", "M&M.NS", "MARUTI.NS", "NESTLEIND.NS",
         "NTPC.NS", "ONGC.NS", "POWERGRID.NS", "RELIANCE.NS", "SBIN.NS",
         "SHREECEM.NS", "SUNPHARMA.NS", "TATACONSUM.NS", "TATAMOTORS.NS",
         "TATASTEEL.NS", "TECHM.NS", "TITAN.NS", "ULTRACEMCO.NS", "WIPRO.NS",
         "UPL.NS", "IOC.NS", "TCS.NS"
     ]

    #return [
    #    "ASIANPAINT.NS", "COALINDIA.NS", "DIVISLAB.NS",
    #    "DRREDDY.NS", "EICHERMOT.NS", "GAIL.NS", "GRASIM.NS", "M&M.NS", "MARUTI.NS",
    #    "NESTLEIND.NS", "POWERGRID.NS", "RELIANCE.NS", "TATACONSUM.NS", "TATAMOTORS.NS",
    #    "TITAN.NS", "ULTRACEMCO.NS", "TCS.NS"
    #]

# Load historical data
stock_data_dict = {}
end_date = date.today() - relativedelta(years=1)
start_date = end_date - relativedelta(years=20)

for ticker in get_nifty50_tickers():
    try:
        data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False, progress=False)
        data = data.rename(columns={
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Adj Close': 'adj_close',
            'Volume': 'volume'
        }).reset_index()
        data['turnover'] = data['volume'] * data['close']
        data = data[['Date', 'open', 'high', 'low', 'close', 'volume', 'turnover']]
        data = data.rename(columns={'Date': 'timestamp'})
        stock_data_dict[ticker] = data.dropna()
    except Exception as e:
        print(f"Failed to process {ticker}: {str(e)}")
        continue

def clean_stock_data(stock_dict):
    """Clean and prepare stock data"""
    cleaned_dict = {}
    for ticker, df in stock_dict.items():
        try:
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel(1)
            df = df.copy()
            df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover']]
            df.set_index('timestamp', inplace=True)
            df.index = pd.to_datetime(df.index)
            df.sort_index(inplace=True)
            df = df[~df.index.duplicated()]
            df.dropna(inplace=True)
            cleaned_dict[ticker] = df
        except Exception as e:
            print(f"Cleaning failed for {ticker}: {str(e)}")
            continue
    return cleaned_dict

cleaned_data = clean_stock_data(stock_data_dict)
print(f"Successfully loaded and cleaned data for {len(cleaned_data)} stocks")

Successfully loaded and cleaned data for 47 stocks


FEATURE ENGINEERING

In [ ]:
# CELL 3: Feature Engineering Functions (Reduced Complexity for Overfitting Control)

# ---- LAW 0a: Volatility Scaling (Absolute Return) Features ----
def roll_abs_return_mean(arr):
    ret = np.diff(arr) / arr[:-1]
    return np.mean(np.abs(ret))

def roll_abs_return_rms(arr):
    ret = np.diff(arr) / arr[:-1]
    return np.sqrt(np.mean(ret**2))

def roll_abs_return_var(arr):
    ret = np.diff(arr) / arr[:-1]
    return np.var(ret)

def roll_abs_return_median(arr):
    ret = np.diff(arr) / arr[:-1]
    return np.median(np.abs(ret))

# Advanced Volatility Features (Law 0a Variants)
def calc_roll_median_abs_return(arr):
    ret = np.diff(arr) / arr[:-1]
    return np.median(np.abs(ret))

def calc_volatility_ratio(df, short_window=5, long_window=20):
    vol_short = df['close'].pct_change().abs().rolling(window=short_window, min_periods=short_window).mean()
    vol_long = df['close'].pct_change().abs().rolling(window=long_window, min_periods=long_window).mean()
    return vol_short / vol_long

# ---- LAW 0b: Directional Change Count Features ----
def directional_change_count(arr, threshold):
    count = 0
    direction = None
    last_extreme = arr[0]
    for price in arr:
        change = (price - last_extreme) / last_extreme
        if direction is None:
            direction = 'up' if change >= 0 else 'down'
        else:
            if direction == 'up' and change <= -threshold:
                count += 1
                direction = 'down'
                last_extreme = price
            elif direction == 'down' and change >= threshold:
                count += 1
                direction = 'up'
                last_extreme = price
    return count

# ---- LAW 1: Tick-Count Scaling Law Approximation ----
def tick_count_approx(high, low, close, tick_threshold=0.0002):
    return (high - low) / (tick_threshold * close)

# ---- LAW 2: Frequency of Price Moves Features ----
def roll_move_count(arr, threshold):
    ret = np.diff(arr) / arr[:-1]
    return np.sum(np.abs(ret) >= threshold)

# ---- LAW 3: Maximum Price Range Features ----
def roll_max_range(high_arr, low_arr):
    return np.max(high_arr) - np.min(low_arr)

def roll_range_std(high_arr, low_arr):
    ranges = high_arr - low_arr
    return np.std(ranges)

# ---- LAW 4: Time Between Moves Features ----
def avg_time_between_moves(arr, threshold):
    ret = np.abs(np.diff(arr) / arr[:-1])
    idx = np.where(ret >= threshold)[0]
    if len(idx) < 2:
        return np.nan
    return np.mean(np.diff(idx))

# ---- KALMAN FILTER FAIR PRICE CALCULATION ----
def calculate_kalman_fair_price(df, process_noise=1e-4, observation_noise=1e-2):
    """
    Calculate Kalman filter fair price for the entire time series

    Parameters:
    - df: DataFrame with 'close' column
    - process_noise: Process noise parameter (smaller = more stable)
    - observation_noise: Observation noise parameter (smaller = more responsive)

    Returns:
    - Series of fair prices
    """
    try:
        # Use log prices for better stability
        log_prices = np.log(df['close'].values).reshape(-1, 1)

        # Initialize Kalman Filter
        kf = KalmanFilter(
            initial_state_mean=log_prices[0],
            n_dim_obs=1,
            n_dim_state=1,
            transition_matrices=np.array([[1.0]]),
            observation_matrices=np.array([[1.0]]),
            transition_covariance=np.array([[process_noise]]),
            observation_covariance=np.array([[observation_noise]])
        )

        # Fit the filter
        state_means, _ = kf.filter(log_prices)

        # Convert back to price space
        fair_prices = np.exp(state_means.flatten())

        return pd.Series(fair_prices, index=df.index)

    except Exception as e:
        print(f"Kalman filter calculation failed: {e}")
        # Fallback to simple moving average
        return df['close'].rolling(window=20, min_periods=1).mean()

print("Feature engineering functions defined successfully!")

Feature engineering functions defined successfully!


In [ ]:
# CELL 4: Simplified Feature Engineering Implementation (Overfitting Reduction)

def feature_engineering_simplified(df):
    """
    Compute simplified features based on scaling laws to reduce overfitting.
    This version uses fewer windows and features compared to the original.
    """
    original_cols = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    df = df.copy()
    df.sort_values('timestamp', inplace=True)

    feature_dict = {}

    # REDUCED WINDOWS FOR OVERFITTING CONTROL
    # Original had many windows, this version uses only essential ones
    windows_0a = [10, 20, 60]  # Reduced from [5, 10, 20, 30, 60, 120]
    windows_0b = [20, 60]      # Reduced from [20, 30, 60]
    windows_1 = [10, 20]       # Reduced from [5, 10, 20, 30, 60]
    windows_2 = [20, 60]       # Reduced from [20, 30, 60]
    windows_3 = [20, 60]       # Reduced from [10, 20, 30, 60, 120]
    windows_4 = [20, 60]       # Reduced from [20, 30, 60]

    # REDUCED THRESHOLDS
    thresholds_0b = [0.02, 0.05]    # Reduced from [0.01, 0.02, 0.03, 0.05]
    thresholds_2 = [0.05, 0.10]     # Reduced from [0.03, 0.05, 0.10, 0.15, 0.20]
    thresholds_4 = [0.05, 0.10]     # Reduced from [0.03, 0.05, 0.10, 0.15]

    # ----- LAW 0a: Volatility Scaling Features (Simplified) -----
    for w in windows_0a:
        feature_dict[f"roll_abs_return_mean_{w}"] = df['close'].rolling(window=w, min_periods=w).apply(lambda x: roll_abs_return_mean(x.values))
        feature_dict[f"roll_abs_return_rms_{w}"] = df['close'].rolling(window=w, min_periods=w).apply(lambda x: roll_abs_return_rms(x.values))

    # Only one volatility ratio instead of multiple
    feature_dict["volatility_ratio_5_20"] = calc_volatility_ratio(df, short_window=5, long_window=20)

    # ----- LAW 0b: Directional Change Count Features (Simplified) -----
    for w in windows_0b:
        for thr in thresholds_0b:
            fname = f"roll_dir_change_{w}_thr_{int(thr*100)}"
            feature_dict[fname] = df['close'].rolling(window=w, min_periods=w).apply(lambda x: directional_change_count(x.values, thr))

    # ----- LAW 1: Tick-Count Approximation (Simplified) -----
    tick_threshold = 0.0002
    df["temp_tick_count"] = (df["high"] - df["low"]) / (tick_threshold * df["close"])
    for w in windows_1:
        fname = f"roll_tick_count_{w}"
        feature_dict[fname] = df["temp_tick_count"].rolling(window=w, min_periods=w).mean()
    df.drop(columns=["temp_tick_count"], inplace=True, errors="ignore")

    # ----- LAW 2: Frequency of Price Moves Features (Simplified) -----
    for w in windows_2:
        for thr in thresholds_2:
            fname = f"roll_move_count_{w}_thr_{int(thr*100)}"
            feature_dict[fname] = df['close'].pct_change().rolling(window=w, min_periods=w).apply(lambda x: np.sum(np.abs(x.values[1:]) >= thr))

    # ----- LAW 3: Maximum Price Range Features (Simplified) -----
    for w in windows_3:
        range_series = df['high'] - df['low']
        fname = f"roll_max_range_{w}"
        feature_dict[fname] = df['high'].rolling(window=w, min_periods=w).max() - df['low'].rolling(window=w, min_periods=w).min()
        fname_std = f"roll_range_std_{w}"
        feature_dict[fname_std] = range_series.rolling(window=w, min_periods=w).std()

    # ----- LAW 4: Time Between Moves Features (Simplified) -----
    for w in windows_4:
        for thr in thresholds_4:
            fname = f"roll_avg_time_between_{w}_thr_{int(thr*100)}"
            feature_dict[fname] = df['close'].rolling(window=w, min_periods=w).apply(lambda x: avg_time_between_moves(x.values, thr))

    # ----- KALMAN FAIR PRICE FEATURE -----
    df['kalman_fair_price'] = calculate_kalman_fair_price(df)

    # Price vs Fair Price ratios
    df['price_to_fair_ratio'] = df['close'] / df['kalman_fair_price']
    df['fair_price_deviation'] = (df['close'] - df['kalman_fair_price']) / df['kalman_fair_price']

    # Add these as features
    feature_dict['price_to_fair_ratio'] = df['price_to_fair_ratio']
    feature_dict['fair_price_deviation'] = df['fair_price_deviation']

    # Create features DataFrame
    features_df = pd.DataFrame(feature_dict)
    df = pd.concat([df, features_df], axis=1)

    # Shift features to avoid lookahead bias
    all_feature_names = list(feature_dict.keys())
    df[all_feature_names] = df[all_feature_names].shift(1)

    # Handle NaN values
    df.dropna(how='all', inplace=True)
    df.ffill(inplace=True)
    df.fillna(0, inplace=True)

    # Create feature mapping
    extended_original_cols = original_cols + ['kalman_fair_price', 'price_to_fair_ratio', 'fair_price_deviation']
    sorted_feature_names = extended_original_cols + sorted(all_feature_names)
    feature_mapping = {col: f"{i+1:03d}" for i, col in enumerate(sorted_feature_names)}
    df.rename(columns=feature_mapping, inplace=True)


    return df, feature_mapping

def apply_feature_engineering_simplified(data_dict):
    """Apply simplified feature engineering to all stocks"""
    fe_data = {}
    feature_mappings = {}

    for ticker, df in data_dict.items():
        print(f"Processing features for {ticker}...")
        engineered_df, mapping = feature_engineering_simplified(df)
        fe_data[ticker] = engineered_df
        feature_mappings[ticker] = mapping

    return fe_data, feature_mappings

# Apply feature engineering to cleaned data
feature_data, feature_mappings = apply_feature_engineering_simplified(cleaned_data)

print("Feature engineering completed!")
display(feature_data.get('TATACONSUM.NS'))
display(feature_mappings.get('TATACONSUM.NS'))

Processing features for ADANIENT.NS...
Processing features for ADANIPORTS.NS...
Processing features for ADANIGREEN.NS...
Processing features for ASIANPAINT.NS...
Processing features for AXISBANK.NS...
Processing features for BAJFINANCE.NS...
Processing features for BAJAJFINSV.NS...
Processing features for BPCL.NS...
Processing features for BRITANNIA.NS...
Processing features for CIPLA.NS...
Processing features for COALINDIA.NS...
Processing features for DIVISLAB.NS...
Processing features for DRREDDY.NS...
Processing features for EICHERMOT.NS...
Processing features for GAIL.NS...
Processing features for GRASIM.NS...
Processing features for HCLTECH.NS...
Processing features for HDFCBANK.NS...
Processing features for HEROMOTOCO.NS...
Processing features for HINDUNILVR.NS...
Processing features for ICICIBANK.NS...
Processing features for INDUSINDBK.NS...
Processing features for INFY.NS...
Processing features for ITC.NS...
Processing features for JSWSTEEL.NS...
Processing features for KOTAK

,002,003,004,005,006,turnover,007,011,010,012,...,026,032,027,033,019,018,021,020,011,010
timestamp,,,,,,,,,,,,,,,,,,,,,
2004-06-18,34.500000,34.790001,33.560001,33.779999,1788450,6.041384e+07,33.779999,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
2004-06-21,35.000000,35.000000,33.255001,33.380001,798410,2.665093e+07,33.579394,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,1.000000
2004-06-22,33.000000,33.599998,32.099998,33.384998,759210,2.534622e+07,33.513609,0.994062,0.994062,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.994062,0.994062
2004-06-23,33.700001,34.240002,33.255001,33.689999,2275760,7.667035e+07,33.559034,0.996162,0.996162,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.996162,0.996162
2004-06-24,34.099998,34.200001,33.410000,33.910000,2036870,6.907026e+07,33.632915,1.003903,1.003903,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.003903,1.003903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-10,1139.000000,1143.449951,1125.650024,1133.050049,1392359,1.577612e+09,1105.450246,1.029986,1.029986,0.013498,...,131.099976,15.298954,240.099976,12.926339,12.0,2.0,28.0,2.0,1.029986,1.029986
2024-06-11,1133.050049,1144.800049,1126.699951,1135.599976,805013,9.141727e+08,1108.283448,1.024967,1.024967,0.012971,...,131.099976,15.439910,240.099976,12.652811,12.0,2.0,28.0,2.0,1.024967,1.024967
2024-06-12,1142.000000,1142.000000,1123.500000,1124.650024,773116,8.694849e+08,1109.830012,1.024648,1.024648,0.010761,...,131.099976,15.555843,198.949951,12.036143,12.0,2.0,28.0,2.0,1.024648,1.024648


{'timestamp': '001',
 'open': '002',
 'high': '003',
 'low': '004',
 'close': '005',
 'volume': '006',
 'kalman_fair_price': '007',
 'price_to_fair_ratio': '011',
 'fair_price_deviation': '010',
 'roll_abs_return_mean_10': '012',
 'roll_abs_return_mean_20': '013',
 'roll_abs_return_mean_60': '014',
 'roll_abs_return_rms_10': '015',
 'roll_abs_return_rms_20': '016',
 'roll_abs_return_rms_60': '017',
 'roll_avg_time_between_20_thr_10': '018',
 'roll_avg_time_between_20_thr_5': '019',
 'roll_avg_time_between_60_thr_10': '020',
 'roll_avg_time_between_60_thr_5': '021',
 'roll_dir_change_20_thr_2': '022',
 'roll_dir_change_20_thr_5': '023',
 'roll_dir_change_60_thr_2': '024',
 'roll_dir_change_60_thr_5': '025',
 'roll_max_range_20': '026',
 'roll_max_range_60': '027',
 'roll_move_count_20_thr_10': '028',
 'roll_move_count_20_thr_5': '029',
 'roll_move_count_60_thr_10': '030',
 'roll_move_count_60_thr_5': '031',
 'roll_range_std_20': '032',
 'roll_range_std_60': '033',
 'roll_tick_count_10':

GENERATING TARGET (STATE SPACE MODEL- KALMAN FILTER BASED)

In [ ]:
#from collections.abc import Iterable
def generate_kalman_target(df, feature_mapping,
                           undervaluation_threshold,
                           confirmation_window,
                           trend_requirement=True):
    """
    Generate binary target based on Kalman fair price instead of future price movements.
    """
    # 1. Extract and validate column names
    close_col      = feature_mapping.get("close")
    fair_price_col = feature_mapping.get("kalman_fair_price")
    deviation_col  = feature_mapping.get("fair_price_deviation")


    # 2. Work on a copy and drop rows missing key data
    df = df.copy()
    df = df.dropna(subset=[close_col, fair_price_col, deviation_col])

    # ensure we get a Series, not accidentally a DataFrame
    raw_dev = df[deviation_col]
    if isinstance(raw_dev, pd.DataFrame):
        # take the first column if multiple returned
        raw_dev = raw_dev.iloc[:, 0]
    raw_dev = raw_dev.astype(float)

    df['is_undervalued'] = raw_dev <= float(undervaluation_threshold)
    df['undervalued_persistent'] = (
        df['is_undervalued']
          .rolling(window=confirmation_window, min_periods=1)
          .sum() >= (confirmation_window // 2)
    )
    df['undervalued_persistent'] = (
        df['is_undervalued']
          .rolling(window=confirmation_window, min_periods=1)
          .sum() >= (confirmation_window // 2)
    )

    # 4. Optional trend & vol filters
    if trend_requirement:
        df['recent_return'] = df[close_col].pct_change(5)
        df['trend_ok'] = df['recent_return'] > -0.10

        df['volatility'] = df[close_col].pct_change().rolling(window=10).std()
        vol_cutoff = df['volatility'].quantile(0.9)
        df['volatility_ok'] = df['volatility'] <= vol_cutoff

        df['target'] = (
            df['undervalued_persistent'] &
            df['trend_ok'] &
            df['volatility_ok']
        ).astype(int)
    else:
        df['target'] = df['undervalued_persistent'].astype(int)

    # 5. Cleanup temporary columns
    drop_cols = ['is_undervalued', 'undervalued_persistent']
    if trend_requirement:
        drop_cols += ['recent_return', 'trend_ok', 'volatility', 'volatility_ok']
    df.drop(columns=drop_cols, inplace=True, errors='ignore')

    # Move target to end
    cols = [c for c in df.columns if c != 'target']
    df = df[cols + ['target']]

    # Handle NaNs in target
    if df['target'].isnull().any():
        df['target'].fillna(0, inplace=True)
        print("Warning: NaNs in 'target' filled with 0.")

    # Diagnostics
    dist = df['target'].value_counts().to_dict()
    ratio = dist.get(1, 0) / len(df)
    # print(f"Target distribution: {dist}")
    # print(f"Positive target ratio: {ratio:.3f}")

    return df


def apply_kalman_target_generation(feature_data_dict, feature_mappings, **kwargs):
    """
    Apply Kalman target generation across all tickers.
    """
    result = {}
    for ticker, df in feature_data_dict.items():
        try:
            print(f"\nProcessing {ticker}...")
            mapping = feature_mappings[ticker]
            result[ticker] = generate_kalman_target(df, mapping, **kwargs)
        except Exception as e:
            print(f"Failed for {ticker}: {e}")
    return result


kalman_target_data = apply_kalman_target_generation(
    feature_data,
    feature_mappings,
    undervaluation_threshold=0.985,
    confirmation_window=5,
    trend_requirement=True
)


Processing ADANIENT.NS...

Processing ADANIPORTS.NS...

Processing ADANIGREEN.NS...

Processing ASIANPAINT.NS...

Processing AXISBANK.NS...

Processing BAJFINANCE.NS...

Processing BAJAJFINSV.NS...

Processing BPCL.NS...

Processing BRITANNIA.NS...

Processing CIPLA.NS...

Processing COALINDIA.NS...

Processing DIVISLAB.NS...

Processing DRREDDY.NS...

Processing EICHERMOT.NS...

Processing GAIL.NS...

Processing GRASIM.NS...

Processing HCLTECH.NS...

Processing HDFCBANK.NS...

Processing HEROMOTOCO.NS...

Processing HINDUNILVR.NS...

Processing ICICIBANK.NS...

Processing INDUSINDBK.NS...

Processing INFY.NS...

Processing ITC.NS...

Processing JSWSTEEL.NS...

Processing KOTAKBANK.NS...

Processing LT.NS...

Processing M&M.NS...

Processing MARUTI.NS...

Processing NESTLEIND.NS...

Processing NTPC.NS...

Processing ONGC.NS...

Processing POWERGRID.NS...

Processing RELIANCE.NS...

Processing SBIN.NS...

Processing SHREECEM.NS...

Processing SUNPHARMA.NS...

Processing TATACONSUM.NS.

TARGET BALANCING, HYPERPARAMETER OPTIMIZATION, FEATURE SELECTION, CROSS VALIDATION

In [ ]:
# CELL 6: Overfitting-Reduced Model Training Pipeline

def balance_target_conservative(X, y, sampling_strategy='auto'):
    """
    Conservative target balancing to avoid overfitting
    """
    y = np.array(y)
    if y.dtype != np.int64:
        y = y.astype(int)

    print(f"Initial class distribution: {np.bincount(y)}")

    # Use less aggressive oversampling
    oversampler = SMOTE(random_state=42, sampling_strategy=sampling_strategy)
    X_resampled, y_resampled = oversampler.fit_resample(X, y)

    print(f"Resampled class distribution: {np.bincount(y_resampled)}")
    return X_resampled, y_resampled

def train_test_split_timeseries(df, test_size=0.3):
    """
    Time-series aware train-test split (no shuffling)
    """
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    if X.shape[0] == 0:
        raise ValueError("Input DataFrame is empty")

    if y.isnull().any():
        raise ValueError("Target contains NaN values")

    # Time-series split (no random shuffling)
    split_idx = int(len(df) * (1 - test_size))
    X_train = X.iloc[:split_idx]
    X_test = X.iloc[split_idx:]
    y_train = y.iloc[:split_idx]
    y_test = y.iloc[split_idx:]

    print(f"Train set: {X_train.shape[0]} samples")
    print(f"Test set: {X_test.shape[0]} samples")
    print(f"Train period: {X_train.index[0]} to {X_train.index[-1]}")
    print(f"Test period: {X_test.index[0]} to {X_test.index[-1]}")

    return X_train, X_test, y_train, y_test

def optimize_hyperparameters_conservative(X_train, y_train, n_trials=30):
    """
    Conservative hyperparameter optimization to reduce overfitting
    """
    def objective(trial):
        # More conservative parameter ranges
        params = {
            "objective": "binary:logistic",
            "booster": "gbtree",
            "n_estimators": trial.suggest_int("n_estimators", 50, 150),  # Reduced from 200
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15),  # Reduced from 0.3
            "max_depth": trial.suggest_int("max_depth", 3, 6),  # Reduced from 12
            "gamma": trial.suggest_float("gamma", 0.1, 2.0),  # Increased minimum
            "subsample": trial.suggest_float("subsample", 0.6, 0.9),  # More conservative
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),  # More conservative
            "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 10.0),  # Increased regularization
            "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 5.0),  # Increased regularization
            "random_state": 42
            # "eval_metric": "logloss"
        }

        # Use TimeSeriesSplit for validation
        tscv = TimeSeriesSplit(n_splits=3)
        scores = []

        for train_idx, val_idx in tscv.split(X_train):
            X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

            model = XGBClassifier(**params)
            model.fit(X_tr, y_tr)
            score = model.score(X_val, y_val)
            scores.append(score)

        return np.mean(scores)

    study = create_study(direction='maximize', sampler=TPESampler())
    study.optimize(objective, n_trials=n_trials)

    print(f"Best validation score: {study.best_value:.4f}")
    return study.best_params


def select_important_features_conservative(X_train, y_train, max_features=25):
    # 0) Identify & drop any columns with list/array cells
    ragged = [
        c for c in X_train.columns
        if X_train[c].apply(lambda x: isinstance(x, (list, np.ndarray))).any()
    ]
    if ragged:
        print("Dropping ragged columns:", ragged)
    X_clean = X_train.drop(columns=ragged)

    # 1) Convert to numpy
    X_num = X_clean.astype(np.float64).to_numpy()
    y     = np.asarray(y_train)

    # 2) Train a regularized XGBoost
    model = XGBClassifier(
        objective="binary:logistic",
        n_estimators=100,
        max_depth=4,
        learning_rate=0.1,
        reg_lambda=2.0,
        reg_alpha=1.0,
        random_state=42
    )
    model.fit(X_num, y)

    # 3) Rehydrate importances into pandas
    feature_names = list(X_clean.columns)
    imp = pd.Series(model.feature_importances_, index=feature_names) \
             .sort_values(ascending=False)

    # 4) Pick top-N
    top_feats = imp.nlargest(max_features).index.tolist()
    selected_idx = [feature_names.index(f) for f in top_feats]

    print(
        f"Selected {len(selected_idx)} features "
        f"out of {X_num.shape[1]} numeric inputs."
    )

    # 5) Plot
    #plt.figure(figsize=(8, 5))
    #imp.nlargest(15).plot.barh()
    #plt.title("Top 15 Feature Importances")
    #plt.xlabel("Importance")
    #plt.tight_layout()
    #plt.show()

    return selected_idx

def train_conservative_model(X_train, y_train, X_val, y_val, best_params):
    """
    Train model with enhanced overfitting protection
    """
    # Add additional conservative settings
    conservative_params = best_params.copy()
    conservative_params.update({
        'eval_metric': 'logloss',
        'early_stopping_rounds': 15,  # More aggressive early stopping
        'verbose': False
    })

    # Prepare DMatrix with feature names for better interpretability
    dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=X_train.columns.tolist())
    dval = xgb.DMatrix(X_val, label=y_val, feature_names=X_train.columns.tolist())

    # Train with early stopping
    model = xgb.train(
        params=conservative_params,
        dtrain=dtrain,
        num_boost_round=300,  # Reduced from 500
        evals=[(dtrain, "train"), (dval, "eval")],
        early_stopping_rounds=15,
        verbose_eval=False
    )

    print(f"Training stopped at iteration: {model.best_iteration}")
    print(f"Best validation score: {model.best_score:.4f}")

    return model

def cross_validate_conservative(X, y, n_splits=5):
    """
    Conservative cross-validation using TimeSeriesSplit
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    scores = []

    for train_idx, val_idx in tscv.split(X):
        X_train_cv = X.iloc[train_idx]
        X_val_cv = X.iloc[val_idx]
        y_train_cv = y.iloc[train_idx]
        y_val_cv = y.iloc[val_idx]

        # Simple model for CV
        model = XGBClassifier(
            n_estimators=100,
            max_depth=4,
            learning_rate=0.1,
            reg_lambda=2.0,
            random_state=42
        )

        model.fit(X_train_cv, y_train_cv)
        score = model.score(X_val_cv, y_val_cv)
        scores.append(score)

    print(f"Cross-validation scores: {[f'{s:.3f}' for s in scores]}")
    print(f"Mean CV score: {np.mean(scores):.3f} (+/- {np.std(scores)*2:.3f})")

    return np.mean(scores)

print(" training pipeline functions defined!")

 training pipeline functions defined!


TRAINING, TESTING, EVALUATION

In [ ]:
# CELL 7: Main Execution Pipeline (Conservative Approach)

def execute_conservative_pipeline(df, ticker_name):
    """
    Execute the complete conservative model pipeline
    """
    #print(f"\n{'='*60}")
    #print(f"EXECUTING CONSERVATIVE PIPELINE FOR {ticker_name}")
    #print(f"{'='*60}")

    # Step 1: Time-series aware train-test split
    #print("\n1. Time-series Train-Test Split...")
    X_train, X_test, y_train, y_test = train_test_split_timeseries(
        df, test_size=0.3
    )

    # Step 2: Conservative target balancing
    #print("\n2. Conservative Target Balancing...")
    X_train_balanced, y_train_balanced = balance_target_conservative(X_train, y_train)

    # Step 3: Train-validation split for model selection
    #print("\n3. Creating Validation Set...")
    val_size = int(0.2 * len(X_train_balanced))
    X_train_final = X_train_balanced[:-val_size]
    X_val = X_train_balanced[-val_size:]
    y_train_final = y_train_balanced[:-val_size]
    y_val = y_train_balanced[-val_size:]

    # Convert to DataFrame for consistency
    X_train_final = pd.DataFrame(X_train_final, columns=X_train.columns)
    X_val = pd.DataFrame(X_val, columns=X_train.columns)
    y_train_final = pd.Series(y_train_final)
    y_val = pd.Series(y_val)

    #print(f"Final train set: {len(X_train_final)} samples")
    #print(f"Validation set: {len(X_val)} samples")

    # Step 4: Conservative feature selection
    #print("\n4. Conservative Feature Selection...")
    important_features_idx = select_important_features_conservative(
        X_train_final, y_train_final, max_features=20  # Reduced from 25
    )

    # Apply feature selection
    feature_names = X_train_final.columns[important_features_idx].tolist()
    X_train_selected = X_train_final.iloc[:, important_features_idx]
    X_val_selected = X_val.iloc[:, important_features_idx]
    X_test_selected = X_test.iloc[:, important_features_idx]

    #print(f"Selected features: {feature_names}")

    # Step 5: Conservative hyperparameter optimization
    #print("\n5. Conservative Hyperparameter Optimization...")
    best_params = optimize_hyperparameters_conservative(
        X_train_selected, y_train_final, n_trials=30  # Reduced trials
    )
    #print(f"Best parameters: {best_params}")

    training_feature_names = list(X_train_selected.columns)

    # Step 6: Train final model
    #print("\n6. Training Conservative Model...")
    final_model = train_conservative_model(
        X_train_selected, y_train_final, X_val_selected, y_val, best_params
    )

    # Step 7: Model evaluation
    #print("\n7. Model Evaluation...")

    # Validation predictions
    dval = xgb.DMatrix(X_val_selected, feature_names=feature_names)
    y_val_proba = final_model.predict(dval)
    y_val_pred = (y_val_proba > 0.5).astype(int)

    # Test predictions
    dtest = xgb.DMatrix(X_test_selected, feature_names=feature_names)
    y_test_proba = final_model.predict(dtest)
    y_test_pred = (y_test_proba > 0.5).astype(int)

    # Training predictions
    dtrain = xgb.DMatrix(X_train_selected, feature_names=feature_names)
    y_train_proba = final_model.predict(dtrain)
    y_train_pred = (y_train_proba > 0.5).astype(int)

# To Print performance metrics remove comments

#    print("\nTEST PERFORMANCE:")
#    print(classification_report(y_test, y_test_pred))

#    print("\nTRAIN PERFORMANCE:")
#    print(classification_report(y_train_final, y_train_pred))

    # Calculate overfitting indicators
#    train_acc = accuracy_score(y_train_final, y_train_pred)
#    val_acc = accuracy_score(y_val, y_val_pred)
#    test_acc = accuracy_score(y_test, y_test_pred)

#    overfitting_ratio = (train_acc - test_acc) / train_acc

#    print(f"\nOVERFITTING ANALYSIS:")
#    print(f"Train Accuracy: {train_acc:.3f}")
#    print(f"Validation Accuracy: {val_acc:.3f}")
#    print(f"Test Accuracy: {test_acc:.3f}")
#    print(f"Overfitting Ratio: {overfitting_ratio:.3f} ({'GOOD' if overfitting_ratio < 0.1 else 'POOR'})")

    # Step 8: Cross-validation
#    print("\n8. Cross-Validation Assessment...")
#    cv_score = cross_validate_conservative(X_train_selected, y_train_final)

    # Visualizations
#    print("\n9. Generating Visualizations...")

    # Confusion Matrix
#    cm = confusion_matrix(y_test, y_test_pred)
#    plt.figure(figsize=(8, 6))
#    ConfusionMatrixDisplay(confusion_matrix=cm).plot()
#    plt.title(f"Confusion Matrix - {ticker_name}")
#    plt.show()

    # ROC Curve
    #fpr_test, tpr_test, _ = roc_curve(y_test, y_test_proba)
    #roc_auc_test = auc(fpr_test, tpr_test)

#    fpr_train, tpr_train, _ = roc_curve(y_train_final, y_train_proba)
#    roc_auc_train = auc(fpr_train, tpr_train)
#
#    plt.figure(figsize=(8, 6))
#    plt.plot(fpr_train, tpr_train, label=f"Train ROC (AUC = {roc_auc_train:.3f})")
#    plt.plot(fpr_test, tpr_test, label=f"Test ROC (AUC = {roc_auc_test:.3f})")
#    plt.plot([0, 1], [0, 1], linestyle="--", label="Random")
#    plt.xlabel("False Positive Rate")
#    plt.ylabel("True Positive Rate")
#    plt.title(f"ROC Curves - {ticker_name}")
#    plt.legend()
#    plt.show()


    # Return results
    results = {
        'model': final_model,
#        'best_params': best_params,
        'feature_names': feature_names,
#        'train_accuracy': train_acc,
#        'validation_accuracy': val_acc,
#        'test_accuracy': test_acc,
#        'cv_score': cv_score,
#        'overfitting_ratio': overfitting_ratio,
#        'roc_auc_train': roc_auc_train,
#        'roc_auc_test': roc_auc_test,
    }

    return results

def run_conservative_pipeline_all_stocks(target_data_dict):
    """
    Run conservative pipeline for all stocks
    """
    all_results = {}

    for ticker, df in target_data_dict.items():
        #print(f"\n\nProcessing {ticker}...")

        try:
            results = execute_conservative_pipeline(df, ticker)
            all_results[ticker] = results

#            print(f"\n{ticker} SUMMARY:")
#            print(f"Final Test Accuracy: {results['test_accuracy']:.3f}")
#            print(f"Overfitting Ratio: {results['overfitting_ratio']:.3f}")
#            print(f"Cross-Validation Score: {results['cv_score']:.3f}")

        except Exception as e:
            #print(f"Error processing {ticker}: {str(e)}")
            continue

    return all_results

# Execute the conservative pipeline
model_results = run_conservative_pipeline_all_stocks(kalman_target_data)

Train set: 3457 samples
Test set: 1482 samples
Train period: 2004-06-18 00:00:00 to 2018-06-11 00:00:00
Test period: 2018-06-12 00:00:00 to 2024-06-14 00:00:00
Initial class distribution: [2304 1153]
Resampled class distribution: [2304 2304]


[I 2025-06-18 14:29:29,037] A new study created in memory with name: no-name-c4012684-0729-4e73-a8a4-c1126939d185


Selected 20 features out of 40 numeric inputs.


[I 2025-06-18 14:29:29,584] Trial 0 finished with value: 0.8450959102424901 and parameters: {'n_estimators': 147, 'learning_rate': 0.07700878665126014, 'max_depth': 5, 'gamma': 0.8883877365548594, 'subsample': 0.749692821557892, 'colsample_bytree': 0.7427790372887386, 'reg_lambda': 5.491235068647209, 'reg_alpha': 3.0047065183970183}. Best is trial 0 with value: 0.8450959102424901.
[I 2025-06-18 14:29:30,204] Trial 1 finished with value: 0.8385812522620341 and parameters: {'n_estimators': 136, 'learning_rate': 0.02494211224677156, 'max_depth': 5, 'gamma': 1.6513723326419174, 'subsample': 0.7352014523427093, 'colsample_bytree': 0.7339560843341445, 'reg_lambda': 6.117718258927624, 'reg_alpha': 2.992021095088864}. Best is trial 0 with value: 0.8450959102424901.
[I 2025-06-18 14:29:30,612] Trial 2 finished with value: 0.8472674629026421 and parameters: {'n_estimators': 108, 'learning_rate': 0.0776767477103432, 'max_depth': 4, 'gamma': 0.554137213014038, 'subsample': 0.8980959032530065, 'col

Best validation score: 0.8494
Training stopped at iteration: 255
Best validation score: 0.1888
Train set: 2853 samples
Test set: 1224 samples
Train period: 2007-11-27 00:00:00 to 2019-07-02 00:00:00
Test period: 2019-07-03 00:00:00 to 2024-06-14 00:00:00
Initial class distribution: [1955  898]
Resampled class distribution: [1955 1955]


...

Best validation score: 0.8594
Training stopped at iteration: 143
Best validation score: 0.2536
Train set: 3457 samples
Test set: 1482 samples
Train period: 2004-06-18 00:00:00 to 2018-06-11 00:00:00
Test period: 2018-06-12 00:00:00 to 2024-06-14 00:00:00
Initial class distribution: [2686  771]
Resampled class distribution: [2686 2686]


[I 2025-06-18 14:42:15,601] A new study created in memory with name: no-name-7263bdc0-a070-4347-bb96-4e6e2e7cfad2


Selected 20 features out of 40 numeric inputs.


[I 2025-06-18 14:42:16,067] Trial 0 finished with value: 0.8336436995654872 and parameters: {'n_estimators': 83, 'learning_rate': 0.09294846456672996, 'max_depth': 6, 'gamma': 0.8853438285445795, 'subsample': 0.8603980056867513, 'colsample_bytree': 0.6473882008783677, 'reg_lambda': 4.961983338741078, 'reg_alpha': 0.5399824634355921}. Best is trial 0 with value: 0.8336436995654872.
[I 2025-06-18 14:42:16,461] Trial 1 finished with value: 0.8479205462445686 and parameters: {'n_estimators': 91, 'learning_rate': 0.06813895944062018, 'max_depth': 5, 'gamma': 1.8475920520405187, 'subsample': 0.7333674699729112, 'colsample_bytree': 0.800684628594813, 'reg_lambda': 7.355438884539827, 'reg_alpha': 3.0042340372756406}. Best is trial 1 with value: 0.8479205462445686.
[I 2025-06-18 14:42:16,764] Trial 2 finished with value: 0.8373680943513345 and parameters: {'n_estimators': 83, 'learning_rate': 0.13361127110500187, 'max_depth': 5, 'gamma': 1.2948909058664915, 'subsample': 0.7408089615928711, 'col

Best validation score: 0.8523
Training stopped at iteration: 34
Best validation score: 0.2319


LIVE PREDICTIONS TO GET A BUY LIST OF STOCKS

In [ ]:
def make_predictions(model, X, feature_names=None):
    """Ensure feature alignment before prediction and drop duplicate columns."""
    # 1) Align to expected feature list if provided
    if feature_names is not None:
        missing = set(feature_names) - set(X.columns)
        if missing:
            raise ValueError(f"Missing features in prediction data: {missing}")
        # enforce order, drop names not in feature_names
        X = X.loc[:, [f for f in feature_names if f in X.columns]]

    # 2) Drop duplicate columns (keep first)
    X = X.loc[:, ~X.columns.duplicated()]

    # 3) Use the final column list as the DMatrix feature names
    final_feat_names = list(X.columns)

    # 4) Convert to numpy and build DMatrix
    X_numeric = X.astype(np.float64).to_numpy()
    dmatrix = xgb.DMatrix(X_numeric, feature_names=final_feat_names)

    # 5) Predict(Can change the inequality with y_pred_proba from 0.5 to filter the weak ones out)
    y_pred_proba = model.predict(dmatrix)
    y_pred       = (y_pred_proba > 0.5).astype(int)
    return y_pred_proba, y_pred

In [ ]:
live_data_dict = {}

for ticker in get_nifty50_tickers():
    try:
        data = yf.download(ticker, period="400d", interval="1d", progress=False)

        data = data.rename(columns={
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Adj Close': 'adj_close',
            'Volume': 'volume'
        }).reset_index()

        data['turnover'] = data['volume'] * data['close']

        data = data[['Date', 'open', 'high', 'low', 'close', 'volume', 'turnover']]
        data = data.rename(columns={'Date': 'timestamp'})

        live_data_dict[ticker] = data.dropna()

    except Exception as e:
        print(f"Failed to process live data for {ticker}: {str(e)}")
        continue

# print(f"\nCollected live data for {len(live_data_dict)} stocks")
# print("Sample live data for CANBK.NS (showing the last few rows to verify latest signal):")
# display(live_data_dict.get('CANBK.NS', pd.DataFrame()).tail())

cleaned_data_2 = clean_stock_data(live_data_dict)

# display(cleaned_data_2.get('POWERGRID.NS', pd.DataFrame()).tail())



def live_prediction(live_data_dict, model_dict):
    """
    Process live data for each stock to get a prediction on the most recent datapoint.

    For each stock in live_data_dict:
      1. Ensure the DataFrame contains at least 50 data points (sorted by timestamp).
      2. Take the last 50 datapoints and apply feature engineering.
      3. Generate target features using generate_simple_target with target_pct=0.03 and drawdown_limit=0.98.
      4. Extract the latest row and align its features with the training features from model_dict.
      5. Use the stored XGBoost model to predict the probability and binary target.
      6. If the predicted label is 1, add the ticker to the buy list.

    Returns:
      buy_list: List of tickers with a BUY signal.
      predictions: Dictionary with ticker as key and (predicted probability, predicted label) as value.
    """
    buy_list = []
    predictions = {}

    for ticker, live_df in live_data_dict.items():

        if len(live_df) < 50:
            print(f"Not enough data for {ticker}. Skipping.")
            continue

        live_df = live_df.sort_index()
        live_df_recent = live_df.iloc[-380:].copy()

        fe_live_df, live_mapping = feature_engineering_simplified(live_df_recent)
        live_df_with_target = generate_kalman_target(fe_live_df, live_mapping, undervaluation_threshold=0.985, confirmation_window=5, trend_requirement=True)

        latest_row = live_df_with_target.iloc[[-1]]

        if ticker not in model_dict:
            print(f"Model for {ticker} not found. Skipping.")
            continue

        training_features = model_dict[ticker]["feature_names"]

        X_live = latest_row[training_features]

        pred_proba, pred = make_predictions(model_dict[ticker]["model"], X_live, feature_names=training_features)
        predictions[ticker] = (pred_proba[0], pred[0])

        if pred[0] == 1:
            buy_list.append(ticker)
            print(f"{ticker}: BUY signal (predicted probability: {pred_proba[0]:.2f})")
        else:
            print(f"{ticker}: NO BUY signal (predicted probability: {pred_proba[0]:.2f})")


    return buy_list, predictions


buy_list, live_predictions = live_prediction(cleaned_data_2, model_results)

<ipython-input-11-3169175658>:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="400d", interval="1d", progress=False)
<ipython-input-11-3169175658>:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="400d", interval="1d", progress=False)
<ipython-input-11-3169175658>:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="400d", interval="1d", progress=False)
<ipython-input-11-3169175658>:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="400d", interval="1d", progress=False)
<ipython-input-11-3169175658>:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="400d", interval="1d", progress=False)
<ipython-input-11-3169175658>:5: FutureWarning: YF.download() has chan

ADANIENT.NS: NO BUY signal (predicted probability: 0.30)
ADANIPORTS.NS: NO BUY signal (predicted probability: 0.26)
ADANIGREEN.NS: NO BUY signal (predicted probability: 0.24)
ASIANPAINT.NS: NO BUY signal (predicted probability: 0.14)
AXISBANK.NS: NO BUY signal (predicted probability: 0.13)
BAJFINANCE.NS: NO BUY signal (predicted probability: 0.02)
BAJAJFINSV.NS: NO BUY signal (predicted probability: 0.23)
BPCL.NS: NO BUY signal (predicted probability: 0.19)
BRITANNIA.NS: NO BUY signal (predicted probability: 0.05)
CIPLA.NS: NO BUY signal (predicted probability: 0.09)
COALINDIA.NS: NO BUY signal (predicted probability: 0.22)
DIVISLAB.NS: NO BUY signal (predicted probability: 0.20)
DRREDDY.NS: NO BUY signal (predicted probability: 0.07)
EICHERMOT.NS: NO BUY signal (predicted probability: 0.12)
GAIL.NS: NO BUY signal (predicted probability: 0.09)
GRASIM.NS: NO BUY signal (predicted probability: 0.18)
HCLTECH.NS: NO BUY signal (predicted probability: 0.04)
HDFCBANK.NS: NO BUY signal (predi

In [ ]:
buy_with_probs = {
    ticker: live_predictions[ticker][0]
    for ticker in buy_list
}

# Sort by probability, highest first
sorted_buys = sorted(
    buy_with_probs.items(),
    key=lambda kv: kv[1],
    reverse=True
)

print("\n=== BUY LIST (sorted by predicted probability) ===")
for ticker, prob in sorted_buys:
    print(f"{ticker:10s} →  {prob*100:5.1f}%")


=== BUY LIST (sorted by predicted probability) ===
TATACONSUM.NS →   90.8%
TATAMOTORS.NS →   84.4%
TITAN.NS   →   74.8%
POWERGRID.NS →   70.4%
SUNPHARMA.NS →   66.7%
ULTRACEMCO.NS →   57.3%
ITC.NS     →   53.4%
SHREECEM.NS →   52.4%


In [ ]:
# ——— 2) Enrich each buy with price/fair‐value info and signal strength ———
enriched_predictions = {}
final_buy_signals = []

for ticker, prob in sorted_buys:
    # grab the latest features DataFrame you used for prediction
    # if you need to recompute it, re‑run feature engineering on the last 1 row:
    latest_features = generate_kalman_target(
        *feature_engineering_simplified(live_data_dict[ticker].iloc[-380:]) , undervaluation_threshold = 0.985 , confirmation_window = 5 , trend_requirement = True
    ).iloc[[-1]]

    # fetch the mapping you used
    _, live_mapping = feature_engineering_simplified(cleaned_data_2[ticker].iloc[-380:])
    close_col      = live_mapping["close"]
    fair_price_col = live_mapping["kalman_fair_price"]
    deviation_col  = live_mapping["fair_price_deviation"]


    # Current price
    raw_curr = latest_features[close_col]
    if isinstance(raw_curr, pd.DataFrame):
        raw_curr = raw_curr.iloc[:, 0]
    current_price = float(raw_curr.iat[0])

    # Fair price
    raw_fair = latest_features[fair_price_col]
    if isinstance(raw_fair, pd.DataFrame):
        raw_fair = raw_fair.iloc[:, 0]
    fair_price = float(raw_fair.iat[0])

    # Deviation
    raw_dev = latest_features[deviation_col]
    if isinstance(raw_dev, pd.DataFrame):
        raw_dev = raw_dev.iloc[:, 0]
    deviation = float(raw_dev.iat[0])

    # compute true percent deviation
    deviation_pct = (fair_price - current_price) / current_price * 100

    # binary prediction label
    _, pred = live_predictions[ticker]

    enriched_predictions[ticker] = {
        'probability':     prob,
        'prediction':      pred,
        'current_price':   current_price,
        'fair_price':      fair_price,
        'deviation_pct':   deviation_pct,
        'is_undervalued':  deviation_pct >= 2.0,
        'signal_strength': 'STRONG' if prob > 0.7 else 'WEAK' if prob > 0.5 else 'NONE'
    }

    if pred == 1:
        final_buy_signals.append(ticker)
        print(f"\n🟢 BUY SIGNAL for {ticker}")
        print(f"   Probability    : {prob:.3f}")
        print(f"   Current Price  : ₹{current_price:.2f}")
        print(f"   Fair Price     : ₹{fair_price:.2f}")
        # Now deviation is a float, so this will format cleanly
        print(f"   Undervaluation : {deviation_pct:.1f}% ({'YES' if deviation_pct >= 2.0 else 'NO'})")
        print(f"   Signal Strength: {enriched_predictions[ticker]['signal_strength']}")
    else:
        print(f"\n🔴 NO BUY signal for {ticker} (prob: {prob:.3f})")


# ——— 3) Generate trading recommendations ———
def generate_trading_recommendations(buy_signals, predictions, capital=100000):
    print("\n" + "="*80)
    print("TRADING RECOMMENDATIONS")
    print("="*80)

    if not buy_signals:
        print("❌ No buy signals generated.")
        return

    print(f"💰 Total Capital: ₹{capital:,.0f}")
    print(f"📈 Buy Signals: {len(buy_signals)} stocks")
    position_per_stock = capital / len(buy_signals)

    for ticker in buy_signals:
        info = predictions[ticker]
        cp = info['current_price']
        shares = int(position_per_stock / cp)
        position_value = shares * cp
        t1 = cp * 1.05
        t2 = cp * 1.10
        sl = cp * 0.95

        print(f"\n🏢 {ticker}")
        print(f"   Signal Strength: {info['signal_strength']}")
        print(f"   Probability    : {info['probability']:.3f}")
        print(f"   Current Price  : ₹{cp:.2f}")
        print(f"   Fair Price     : ₹{info['fair_price']:.2f}")
        print(f"   Undervaluation : {info['deviation_pct']:.1f}%")
        print(f"   Shares to Buy  : {shares:,}")
        print(f"   Position Value : ₹{position_value:,.0f}")
        print(f"   Target 1 (5%)  : ₹{t1:.2f}")
        print(f"   Target 2 (10%) : ₹{t2:.2f}")
        print(f"   Stop Loss (5%): ₹{sl:.2f}")
        print("-" * 50)

# finally call it:
capital = float(input("\nEnter trading capital (₹): ") or "100000")
generate_trading_recommendations(final_buy_signals, enriched_predictions, capital)


🟢 BUY SIGNAL for TATACONSUM.NS
   Probability    : 0.908
   Current Price  : ₹1065.20
   Fair Price     : ₹1101.04
   Undervaluation : 3.4% (YES)
   Signal Strength: STRONG

🟢 BUY SIGNAL for TATAMOTORS.NS
   Probability    : 0.844
   Current Price  : ₹670.25
   Fair Price     : ₹701.55
   Undervaluation : 4.7% (YES)
   Signal Strength: STRONG

🟢 BUY SIGNAL for TITAN.NS
   Probability    : 0.748
   Current Price  : ₹3467.80
   Fair Price     : ₹3490.53
   Undervaluation : 0.7% (NO)
   Signal Strength: STRONG

🟢 BUY SIGNAL for POWERGRID.NS
   Probability    : 0.704
   Current Price  : ₹287.30
   Fair Price     : ₹292.85
   Undervaluation : 1.9% (NO)
   Signal Strength: STRONG

🟢 BUY SIGNAL for SUNPHARMA.NS
   Probability    : 0.667
   Current Price  : ₹1648.00
   Fair Price     : ₹1685.60
   Undervaluation : 2.3% (YES)
   Signal Strength: WEAK

🟢 BUY SIGNAL for ULTRACEMCO.NS
   Probability    : 0.573
   Current Price  : ₹11406.00
   Fair Price     : ₹11384.28
   Undervaluation : -0.2% (

In [ ]:
data_2 = {}
for ticker in buy_list:
    df = yf.download(ticker, period="400d", interval="1d", progress=False)
    df = df.rename(columns={
        'Open':'open','High':'high','Low':'low','Close':'close',
        'Adj Close':'adj_close','Volume':'volume'
    }).reset_index()
    df['turnover'] = df['volume'] * df['close']
    df = df[['Date','open','high','low','close','volume','turnover']]
    df.rename(columns={'Date':'timestamp'}, inplace=True)
    data_2[ticker] = df.dropna().reset_index(drop=True)

cleaned_data_2= clean_stock_data(data_2)

window = 380
target_pct = 0.15
drawdown_limit = 0.95
#can merge with below cell
#data_2['TITAN.NS']--------just checking

<ipython-input-14-2209754833>:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="400d", interval="1d", progress=False)
<ipython-input-14-2209754833>:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="400d", interval="1d", progress=False)
<ipython-input-14-2209754833>:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="400d", interval="1d", progress=False)
<ipython-input-14-2209754833>:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="400d", interval="1d", progress=False)
<ipython-input-14-2209754833>:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="400d", interval="1d", progress=False)
<ipython-input-14-2209754833>:3: FutureWarning: YF.download() has changed argume

In [ ]:
ticker = input("\nENTER A TICKER FROM THE BUY LIST FOR MORE TRADE PREDICTIONS: ").strip()

if ticker not in buy_list:
    print(f"❌ '{ticker}' is not in the BUY list. Please try again.")
else:
    df = cleaned_data_2[ticker].copy()
    if len(df) < window:
        print(f"{ticker}: not enough history ({len(df)} < {window})")
    else:
        live_df = df.iloc[-window:].copy()
        live_df = live_df.sort_index()

        sig_data = pd.DataFrame(index=live_df.index)

        # Kalman fair value
        live_df['log_close'] = np.log(live_df['close'])
        kf = KalmanFilter(initial_state_mean=live_df['log_close'].iloc[0], n_dim_obs=1, n_dim_state=1)
        state_means, _ = kf.filter(live_df['log_close'].values.reshape(-1,1))
        sig_data['fair_price'] = np.exp(state_means.flatten())
        sig_data['close'] = live_df['close'].values

        # Signal logic
        sig_data['strat_signal'] = np.where(sig_data['close'] < sig_data['fair_price'], 1, 0)
        updated, in_trade, entry_price = [], False, np.nan

        for i in sig_data.index:
            if not in_trade and sig_data.at[i, 'strat_signal'] == 1:
                in_trade, entry_price = True, sig_data.at[i, 'close']
                updated.append(1)
            elif in_trade:
                price = sig_data.at[i, 'close']
                if price >= (1 + target_pct) * entry_price or price <= drawdown_limit * entry_price:
                    in_trade = False
                    updated.append(0)
                else:
                    updated.append(1)
            else:
                updated.append(0)

        sig_data['strat_signal'] = pd.Series(updated, index=sig_data.index).shift(1).fillna(0).astype(int)

        # Strategy & Benchmark returns
        returns_series = live_df['close'].pct_change().fillna(0).values
        strat_returns = pd.Series(sig_data['strat_signal'].values * returns_series, index=live_df.index)

        bench_strat = df['close'].pct_change()
        bench_strat.name = "returns"
        bench_strat = bench_strat.to_frame()
        bench_strat.dropna(inplace=True)

        # Vol targeting
        tgt_vol = 0.15
        signallls = pd.DataFrame()
        signallls['stdev'] = (bench_strat['returns']).rolling(22).std() * np.sqrt(252)
        signallls['vol_tgt'] = tgt_vol / signallls['stdev']
        signallls['vol_tgt'] = signallls['vol_tgt'].clip(0, 2)
        volatility_tp = signallls.iloc[-window:]

        vol_signal = sig_data['strat_signal'].values * volatility_tp['vol_tgt']
        vol_strat_returns = pd.Series(vol_signal.values * returns_series, index=live_df.index)

        # QuantStats
        import quantstats as qs
        qs.reports.metrics(
            returns=vol_strat_returns,
            benchmark=bench_strat['returns'],
            mode='full'
        )


ENTER A TICKER FROM THE BUY LIST FOR MORE TRADE PREDICTIONS: TATACONSUM.NS
                           Benchmark    Strategy
-------------------------  -----------  ----------
Start Period               2023-12-06   2023-12-06
End Period                 2025-06-18   2025-06-18
Risk-Free Rate             0.0%         0.0%
Time in Market             99.0%        96.0%

Cumulative Return          14.14%       24.21%
CAGR﹪                     6.13%        10.25%

Sharpe                     0.48         1.04
Prob. Sharpe Ratio         72.21%       90.0%
Smart Sharpe               0.48         1.03
Sortino                    0.71         1.63
Smart Sortino              0.71         1.62
Sortino/√2                 0.5          1.15
Smart Sortino/√2           0.5          1.15
Omega                      1.2          1.2

Max Drawdown               -29.01%      -17.49%
Longest DD Days            465          464
Volatility (ann.)          24.68%       15.03%
R^2                        0.87     